# Linear Algebra of Golf Ball Rotation

This notebook demonstrates how 2-D images of a golf ball can be 
projected onto a 3-D model in order to analyze its rotation.
We use linear algebra concepts such as projection and rotation matrices 
and leverage the `ProjectOp` utility from this repository.

## 1. Mapping 2‑D Pixels to the Ball Surface

Given a detected ball center `(c_x, c_y)` and radius `r`, any pixel `(x, y)`
within the circle can be mapped to 3‑D coordinates on the hemisphere:

$$x' = x - c_x,\quad y' = y - c_y$$
$$z' = \sqrt{r^2 - x'^2 - y'^2}$$

This yields a point **p** = `(x', y', z')` on the ball surface.

In [ ]:
import numpy as np

def map_to_ball(x, y, c_x, c_y, r):
    x_p = x - c_x
    y_p = y - c_y
    z_sq = r*r - (x_p**2 + y_p**2)
    if z_sq <= 0:
        return None  # off the sphere
    z_p = np.sqrt(z_sq)
    return np.array([x_p, y_p, z_p])

# Example pixel
pt_3d = map_to_ball(110, 95, c_x=100, c_y=100, r=30)
pt_3d

## 2. Applying a Rotation

A rotation matrix `R` transforms the 3‑D point on the ball.
Using XYZ Euler angles $lpha$, $eta$, $\gamma$ (side‑spin, back‑spin, axial‑spin):

$$R = R_z(\gamma) R_y(eta) R_x(lpha)$$

The rotated point is $p_r = R p$.

In [ ]:
def euler_rotate(pt, angles_deg):
    ax, ay, az = np.deg2rad(angles_deg)
    Rx = np.array([[1, 0, 0],
                   [0, np.cos(ax), -np.sin(ax)],
                   [0, np.sin(ax),  np.cos(ax)]])
    Ry = np.array([[ np.cos(ay), 0, np.sin(ay)],
                   [0, 1, 0],
                   [-np.sin(ay), 0, np.cos(ay)]])
    Rz = np.array([[np.cos(az), -np.sin(az), 0],
                   [np.sin(az),  np.cos(az), 0],
                   [0, 0, 1]])
    R = Rz @ Ry @ Rx
    return R @ pt

rotated = euler_rotate(pt_3d, (10, 20, 30))
rotated

## 3. Using `ProjectOp`

`ProjectOp` encapsulates the above mapping and rotation steps.
It projects each pixel in a grayscale image to the ball surface,
rotates the point in 3‑D, and samples the new 2‑D position.
This is useful for comparing two frames to estimate spin.

In [ ]:
from GolfBall import GolfBall
from ProjectOp import ProjectionOp

ball = GolfBall(x=100, y=100, measured_radius_pixels=30)
output = np.zeros((200, 200, 2), dtype=np.int32)
op = ProjectionOp(ball, output, np.radians(-10), np.radians(20), np.radians(30))

# project a single pixel value onto the rotated ball
op(255, 110, 95)
output[95, 110]

## 4. Recovering Rotation

By comparing how features move between two frames, we search for
the Euler angles that best align the projected images.
`GetBallRotation.py` automates this search over a grid of rotations.

## 5. Demo: Rotating a Sample Image

Load a grayscale image, rotate it using `get_rotated_image`, and display the result.

In [ ]:
import cv2
import matplotlib.pyplot as plt
from GolfBall import GolfBall
from GetRotatedImage import get_rotated_image

img = cv2.imread("data/Images/image.png", cv2.IMREAD_GRAYSCALE)
ball = GolfBall(x=img.shape[1]//2, y=img.shape[0]//2,
                measured_radius_pixels=min(img.shape)//4,
                angles_camera_ortho_perspective=(0,0,0))
rot_img = get_rotated_image(img, ball, (20, 30, 45))

fig, axes = plt.subplots(1, 2, figsize=(8,4))
axes[0].imshow(img, cmap='gray')
axes[0].set_title('Original')
axes[0].axis('off')
axes[1].imshow(rot_img, cmap='gray')
axes[1].set_title('Rotated')
axes[1].axis('off')
plt.show()